In [7]:
import pandas as pd
import numpy as np
import h5py
import itertools

In [ ]:
samples = pd.read_csv('Reviews.csv')
max = samples['Review'].apply(len).max()

max

In [24]:
unique = sorted(list(set(itertools.chain.from_iterable(samples['Review']))), key=lambda x: ord(x[0]))
vocabulary = {"<EOS>":0,"<PAD>":1,"<OOV>":2} | {unique[i]:i+3 for i in range(len(unique))}

len(vocabulary)

209

In [22]:
samples['x'] = samples['Review'].apply(lambda x: [vocabulary.get(c, 3) for c in x])
samples['x'] = samples['x'].apply(lambda x: (x+[0]+([1]*max))[:max])

samples['x']

0        [60, 84, 87, 80, 78, 83, 76, 5, 92, 78, 89, 77...
1        [60, 74, 81, 81, 19, 19, 19, 89, 87, 74, 82, 8...
2        [52, 90, 72, 77, 6, 5, 57, 77, 78, 88, 5, 84, ...
3        [46, 12, 91, 74, 5, 88, 74, 74, 83, 5, 88, 84,...
4        [7, 40, 70, 87, 87, 78, 74, 87, 88, 7, 5, 75, ...
                               ...                        
49995    [57, 77, 78, 88, 5, 82, 84, 91, 78, 74, 5, 78,...
49996    [51, 78, 72, 74, 5, 89, 84, 5, 88, 74, 74, 5, ...
49997    [47, 74, 70, 83, 5, 55, 74, 83, 84, 78, 87, 12...
49998    [60, 77, 70, 89, 5, 70, 87, 74, 5, 89, 77, 74,...
49999    [46, 5, 88, 70, 92, 5, 89, 77, 78, 88, 5, 82, ...
Name: x, Length: 50000, dtype: object

In [23]:
samples['y'] = samples['Review'].str.lower()
samples['y'] = samples['y'].str.replace(r'[^a-z\s\']',' ',regex=True)
samples['y'] = samples['y'].str.replace(r'\'', '', regex=True)
samples['y'] = samples['y'].str.replace(r'^\s+','',regex=True)
samples['y'] = samples['y'].str.replace(r'\s+$','',regex=True)
samples['y'] = samples['y'].str.replace(r'\s+',' ',regex=True)

samples['y'] = samples['y'].apply(lambda x: [vocabulary.get(c, 3) for c in x])
samples['y'] = samples['y'].apply(lambda x: (x+[0]+([1]*max))[:max])

samples['y']

0        [92, 84, 87, 80, 78, 83, 76, 5, 92, 78, 89, 77...
1        [92, 74, 81, 81, 5, 89, 87, 74, 82, 84, 87, 88...
2        [84, 90, 72, 77, 5, 89, 77, 78, 88, 5, 84, 83,...
3        [78, 91, 74, 5, 88, 74, 74, 83, 5, 88, 84, 82,...
4        [72, 70, 87, 87, 78, 74, 87, 88, 5, 75, 84, 81...
                               ...                        
49995    [89, 77, 78, 88, 5, 82, 84, 91, 78, 74, 5, 78,...
49996    [83, 78, 72, 74, 5, 89, 84, 5, 88, 74, 74, 5, ...
49997    [79, 74, 70, 83, 5, 87, 74, 83, 84, 78, 87, 88...
49998    [92, 77, 70, 89, 5, 70, 87, 74, 5, 89, 77, 74,...
49999    [78, 5, 88, 70, 92, 5, 89, 77, 78, 88, 5, 82, ...
Name: y, Length: 50000, dtype: object

In [25]:
x = np.array(samples['x'].tolist()).astype(np.uint8)
x

array([[60, 84, 87, ...,  1,  1,  1],
       [60, 74, 81, ...,  1,  1,  1],
       [52, 90, 72, ...,  1,  1,  1],
       ...,
       [47, 74, 70, ...,  1,  1,  1],
       [60, 77, 70, ...,  1,  1,  1],
       [46,  5, 88, ...,  1,  1,  1]], dtype=uint8)

In [26]:
y = np.array(samples['y'].tolist()).astype(np.uint8)
y

array([[92, 84, 87, ...,  1,  1,  1],
       [92, 74, 81, ...,  1,  1,  1],
       [84, 90, 72, ...,  1,  1,  1],
       ...,
       [79, 74, 70, ...,  1,  1,  1],
       [92, 77, 70, ...,  1,  1,  1],
       [78,  5, 88, ...,  1,  1,  1]], dtype=uint8)

In [27]:
file = h5py.File('bpe_xy.h5', 'w')

file.create_dataset('x', data=x)
file.create_dataset('y', data=y)

file.close()